<a href="https://colab.research.google.com/github/marco-siino/gpt_networking/blob/main/Q2/Mistral_7B_Instruct_v0_3_LogKnowledgeExtraction_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with `mistral-inference`

This notebook will guide you through the process of running Mistral models locally. We will cover the following:
- How to chat with Mistral 7B Instruct
- How to run Mistral 7B Instruct with function calling capabilities

We recommend using a GPU such as the A100 to run this notebook.

In [ ]:
!pip install mistral-inference

## Download Mistral 7B Instruct

In [ ]:
!wget https://models.mistralcdn.com/mistral-7b-v0-3/mistral-7B-Instruct-v0.3.tar

In [ ]:
!DIR=$HOME/mistral_7b_instruct_v3 && mkdir -p $DIR && tar -xf mistral-7B-Instruct-v0.3.tar -C $DIR

In [ ]:
!ls mistral_7b_instruct_v3

# Import libraries and load the model.

In [1]:
import os
import random

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

# I can decide which GPU to use on this node on Leonardo.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# load tokenizer
mistral_tokenizer = MistralTokenizer.from_file(os.path.expanduser("~")+"/mistral_7b_instruct_v3/tokenizer.model.v3")

# load model
model = Transformer.from_folder(os.path.expanduser("~")+"/mistral_7b_instruct_v3")

In [2]:
!nvidia-smi

Thu Jun 13 12:47:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM-64GB            On | 00000000:1D:00.0 Off |                    0 |
| N/A   43C    P0               77W / 478W|  15129MiB / 65536MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Organize the dataset.

In [3]:
!unzip prompt_Q2-2024-06-13.zip

Archive:  prompt_Q2-2024-06-13.zip
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1046.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=992.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=966.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=982.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1084.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1012.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1086.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1050.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1076.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1070.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1062.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1042.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1010.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1058.txt  
  inflating: prompt_Q2/prompt_n_hours=2_skip_hours=1088.txt  
  inflating: prompt_Q2/prompt_n_hours=

In [20]:
folder_path="prompt_Q2"
# List all files in the folder
files = os.listdir(folder_path)

# Sort the files alphabetically
files.sort()

# Function to pre-process the few shots.

In [13]:
# The following function add the Mistral Tags within the file content.
def preprocess_sample(file_contents):

                sample ="<s>[INST]Given the following LOG, which dev_eui have retransmitted the same frame at least once?\n\nLOG:\n"
                modified_string = file_contents.replace("QUESTION: Which dev_eui have retransmitted the same frame at least once in 2 hours?", "")
                modified_string = modified_string.replace("\n\nANSWER:", "ANSWER:[/INST]")
                modified_string = modified_string[:-1]

                sample += modified_string+"</s>"
                sample += "\n\n"

                return sample


# Create the few-shots samples.

In [21]:
def create_few_shots_set(folder_path,nr_samples):

    # Counter to keep track of the number of files read
    count = 0

    few_shots_set = ""

    # Get a list of all files in the directory
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    # Shuffle the list of files. Seed set - equal to 4 - for reproducibility.
    random.Random(4).shuffle(files)

    global test_set
    test_set = files

    # Iterate through all files in the folder
    for filename in files:
        print(filename)
        # Check if the file is a text file
        if filename.endswith(".txt"):
            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                few_shots_set+=preprocess_sample(file_contents)

                # Increment the counter
                count += 1

                # Check if reached the limit of n files
                if count == n:
                    break
        test_set.remove(filename)

    return few_shots_set

In [22]:
# Specify the number of files to read
n = 5  # Example: Read the first 5 files

# Call the function to read text files in the folder and concatenate their contents
few_shots_samples = create_few_shots_set(folder_path, n)

# Print or use the concatenated contents as needed
print(few_shots_samples)

prompt_n_hours=2_skip_hours=42.txt
prompt_n_hours=2_skip_hours=480.txt
prompt_n_hours=2_skip_hours=1016.txt
prompt_n_hours=2_skip_hours=786.txt
prompt_n_hours=2_skip_hours=348.txt
<s>[INST]Given the following LOG, which dev_eui have retransmitted the same frame at least once?

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size

# Chat with the model to get the results.

In [31]:
# Define the directory path containing the results.
folder_name = 'mistral_7b_Q2_results'
directory_path = os.path.join(os.getcwd(), folder_name)

# Iterate through all the test_files
for filename in test_set:
        print(filename)
        if(os.path.isfile(directory_path+"/"+filename)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Create the directory if it doesn't exist
            if not os.path.exists(directory_path):
                os.makedirs(directory_path)

            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                file_contents= file_contents.replace("\nQUESTION: Which dev_eui have retransmitted the same frame at least once in 2 hours?\n", "")


                prompt = few_shots_samples + "\n[INST]Given the following LOG, which dev_eui have retransmitted the same frame at least once?\n\nLOG:\n"

                # Split the string at the occurrence of "ANSWER:"
                parts = file_contents.split("ANSWER:", 1)

                # Take only until ANSWER
                prompt+= parts[0]+"ANSWER: [/INST]"

                print(prompt)

                # chat completion request
                completion_request = ChatCompletionRequest(messages=[UserMessage(content=prompt)])
                # encode message
                tokens = mistral_tokenizer.encode_chat_completion(completion_request).tokens
                # generate results
                out_tokens, _ = generate([tokens], model, max_tokens=1200, temperature=0.0, eos_id=mistral_tokenizer.instruct_tokenizer.tokenizer.eos_id)
                # decode generated tokens
                result = mistral_tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])
                print(result+"\n\n")

                # Define the directory path
                folder_name = 'mistral_7b_Q2_results'
                directory_path = os.path.join(os.getcwd(), folder_name)

                # Create the directory if it doesn't exist
                if not os.path.exists(directory_path):
                    os.makedirs(directory_path)

                # Define the file path
                file_path = os.path.join(directory_path, filename)

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

prompt_n_hours=2_skip_hours=36.txt
Already done!


prompt_n_hours=2_skip_hours=1082.txt
Already done!


prompt_n_hours=2_skip_hours=748.txt
Already done!


prompt_n_hours=2_skip_hours=100.txt
Already done!


prompt_n_hours=2_skip_hours=348.txt
Already done!


prompt_n_hours=2_skip_hours=140.txt
Already done!


prompt_n_hours=2_skip_hours=122.txt
Already done!


prompt_n_hours=2_skip_hours=220.txt
Already done!


prompt_n_hours=2_skip_hours=578.txt
Already done!


prompt_n_hours=2_skip_hours=402.txt
Already done!


prompt_n_hours=2_skip_hours=706.txt
Already done!


prompt_n_hours=2_skip_hours=580.txt
Already done!


prompt_n_hours=2_skip_hours=240.txt
Already done!


prompt_n_hours=2_skip_hours=426.txt
Already done!


prompt_n_hours=2_skip_hours=1078.txt
Already done!


prompt_n_hours=2_skip_hours=304.txt
Already done!


prompt_n_hours=2_skip_hours=450.txt
Already done!


prompt_n_hours=2_skip_hours=798.txt
Already done!


prompt_n_hours=2_skip_hours=640.txt
Already done!


prompt_n_ho

In [32]:
!zip -r mistral_7b_Q2_results.zip mistral_7b_Q2_results/

  adding: mistral_7b_Q2_results/ (stored 0%)
  adding: mistral_7b_Q2_results/prompt_n_hours=2_skip_hours=478.txt (deflated 45%)
  adding: mistral_7b_Q2_results/prompt_n_hours=2_skip_hours=718.txt (stored 0%)
  adding: mistral_7b_Q2_results/prompt_n_hours=2_skip_hours=310.txt (deflated 28%)
  adding: mistral_7b_Q2_results/prompt_n_hours=2_skip_hours=634.txt (deflated 28%)
  adding: mistral_7b_Q2_results/prompt_n_hours=2_skip_hours=464.txt (deflated 28%)
  adding: mistral_7b_Q2_results/prompt_n_hours=2_skip_hours=760.txt (deflated 27%)
  adding: mistral_7b_Q2_results/prompt_n_hours=2_skip_hours=324.txt (deflated 45%)
  adding: mistral_7b_Q2_results/prompt_n_hours=2_skip_hours=828.txt (deflated 45%)
  adding: mistral_7b_Q2_results/prompt_n_hours=2_skip_hours=144.txt (deflated 54%)
  adding: mistral_7b_Q2_results/prompt_n_hours=2_skip_hours=582.txt (deflated 41%)
  adding: mistral_7b_Q2_results/prompt_n_hours=2_skip_hours=304.txt (deflated 45%)
  adding: mistral_7b_Q2_results/prompt_n_hour

# Given the same few-shot samples ask to Mistral to generate the Python code to perform the task.

In [52]:
prompt ="""
Given the following string of a LOG, write a Python function to OUTPUT (as in the example below) which dev_eui has retransmitted the same frame at least once.

LOG:
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889


OUTPUT:
dev_eui: 121
dev_eui: 124


"""

In [53]:
# chat completion request
completion_request = ChatCompletionRequest(messages=[UserMessage(content=prompt)])
# encode message
tokens = mistral_tokenizer.encode_chat_completion(completion_request).tokens
# generate results
out_tokens, _ = generate([tokens], model, max_tokens=3000, temperature=0.0, eos_id=mistral_tokenizer.instruct_tokenizer.tokenizer.eos_id)
# decode generated tokens
result = mistral_tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])
print(result+"\n\n")

To achieve this, you can create a Python function that stores the `dev_eui` and their corresponding `FCnt` in a dictionary. Then, iterate through the log and check if the current `dev_eui` and `FCnt` already exist in the dictionary. If they do, it means the device has retransmitted the same frame, and you can add it to the output. Here's the code:

```python
def find_retransmitted_frames(log):
    dev_eui_fcnt = {}

    for line in log.split('\n'):
        data = line.split(', ')
        dev_eui = data[10]
        fcnt = int(data[9])

        if dev_eui in dev_eui_fcnt and dev_eui_fcnt[dev_eui] == fcnt:
            print(f"dev_eui: {dev_eui}")

        dev_eui_fcnt[dev_eui] = fcnt if dev_eui not in dev_eui_fcnt else dev_eui_fcnt[dev_eui]

    return dev_eui_fcnt

log = """
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, r

In [54]:
# Let's write a function to use this code.
# To be adherent as much as possible to the original question we try to not change almost anything.
def find_retransmitted_frames(log):
    dev_eui_fcnt = {}

    for line in log.split('\n'):
        data = line.split(', ')
        dev_eui = data[10]
        fcnt = int(data[9])

        if dev_eui in dev_eui_fcnt and dev_eui_fcnt[dev_eui] == fcnt:
            print(f"dev_eui: {dev_eui}")

        dev_eui_fcnt[dev_eui] = fcnt if dev_eui not in dev_eui_fcnt else dev_eui_fcnt[dev_eui]

    return dev_eui_fcnt

log = """
time: 2019-05-21 18:16:47.648000, freq: 868.5, chan: 23, lsnr: 5.0, rssi: -109.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4130
time: 2019-05-21 18:19:00.647000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4142
time: 2019-05-21 18:25:56.343000, freq: 868.1, chan: 21, lsnr: -2.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:02.544000, freq: 868.1, chan: 21, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:26:08.678000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3886
time: 2019-05-21 18:38:09.529000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -113.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 23, FCnt: 3958
time: 2019-05-21 18:39:57.539000, freq: 868.3, chan: 22, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:40:03.735000, freq: 868.3, chan: 22, lsnr: 0.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4137
time: 2019-05-21 18:46:42.609000, freq: 868.5, chan: 23, lsnr: 15.0, rssi: -92.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4088
time: 2019-05-21 18:48:29.241000, freq: 868.1, chan: 21, lsnr: 14.0, rssi: -90.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 186
time: 2019-05-21 19:16:46.731000, freq: 868.3, chan: 22, lsnr: 5.0, rssi: -108.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 126, size: 23, FCnt: 4131
time: 2019-05-21 19:19:00.174000, freq: 868.5, chan: 23, lsnr: -8.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 122, size: 23, FCnt: 4143
time: 2019-05-21 19:25:55.447000, freq: 868.3, chan: 22, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:01.582000, freq: 868.5, chan: 23, lsnr: -5.0, rssi: -115.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:26:07.785000, freq: 868.5, chan: 23, lsnr: -7.0, rssi: -116.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 124, size: 23, FCnt: 3887
time: 2019-05-21 19:38:09.158000, freq: 868.5, chan: 23, lsnr: -1.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 123, size: 25, FCnt: 3959
time: 2019-05-21 19:39:56.692000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:40:02.892000, freq: 868.1, chan: 21, lsnr: 3.0, rssi: -112.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 121, size: 23, FCnt: 4138
time: 2019-05-21 19:46:41.701000, freq: 868.5, chan: 23, lsnr: 8.0, rssi: -106.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 127, size: 25, FCnt: 4089
time: 2019-05-21 19:48:28.310000, freq: 868.1, chan: 21, lsnr: 13.0, rssi: -87.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 120, size: 23, FCnt: 187
time: 2019-05-21 20:12:05.544000, freq: 868.5, chan: 23, lsnr: 0.0, rssi: -114.0, SF: SF7, BW: 125, codr: 4/5, dev_eui: 128, size: 23, FCnt: 3889
"""

find_retransmitted_frames(log)

IndexError: list index out of range

In [39]:
for filename in test_set:
        file_path = folder_name+filename
        with open(os.path.join(folder_path, filename), 'r') as file:
          file_contents = file.read()
          find_duplicate_frames(log_data)
        break

ValueError: not enough values to unpack (expected 2, got 0)

## Let's run the code.

In [ ]:
!rm -r 'mistral_7b_source_code_results/'

In [ ]:
# Define the directory path containing the results.
folder_name = 'mistral_7b_Q2_source_code_results/'
#directory_path = os.path.join(os.getcwd(), folder_name)
# Create the directory if it doesn't exist
if not os.path.exists(folder_name):
  os.makedirs(folder_name)

# Iterate through all the test_files
for filename in test_set:
        file_path = folder_name+filename
        print(filename)
        print(file_path)
        if(os.path.isfile(file_path)):
          print("Already done!\n\n")
          continue
        # Check if the file is a text file
        if filename.endswith(".txt"):

            # Open the file and read its contents
            with open(os.path.join(folder_path, filename), 'r') as file:
                # Read the contents of the file and append it to all_contents
                file_contents = file.read()

                # Remove the ANSWER in the original log file.
                file_contents = file_contents.split("ANSWER", 1)

                result = process_log(file_contents[0])

                print(result+"\n\n")

                with open(file_path, 'w') as f:
                  # Write some content to the file
                  f.write(result)

Streaming output truncated to the last 5000 lines.
For DEVICE dev_eui:128 all received frames are 4.


prompt_n_hours=2_skip_hours=268.txt
mistral_7b_source_code_results/prompt_n_hours=2_skip_hours=268.txt
For DEVICE dev_eui:120 all received frames are 4.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 4.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 4.


prompt_n_hours=2_skip_hours=924.txt
mistral_7b_source_code_results/prompt_n_hours=2_skip_hours=924.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 1.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 3.
For DEVICE dev_eui:124 all received frames are 1.
For DEVICE dev_eui:125 all received fra

# Here the function to run the evaluation at file-level and at global-level.

In [40]:
def evaluate_results (dataset_path,result_path):
  dataset_corpus = ""
  results_corpus = ""
  # Iterate through all the test_files
  for filename in test_set:
      # Specify the paths to the files
      file1_path = result_path+filename
      file2_path = dataset_path+filename

      # Read the contents of the files
      try:
        with open(file1_path, 'r', encoding='utf-8') as file1:
          text1 = file1.read()
      except:
        continue
      try:
        with open(file2_path, 'r', encoding='utf-8') as file2:
          text2 = file2.read()
      except:
        continue

      # Find the index where "ANSWER:" starts
      answer_start_index = text2.find("ANSWER:")

      # Remove all content before and excluding "ANSWER:"
      if answer_start_index != -1:
          text2 = text2[answer_start_index + len("ANSWER:"):].strip()

      print("Sample: ", filename)

      print(text2)
      print("\n")
      print(text1)

      dataset_corpus += text2
      results_corpus += text1

      # Tokenize the texts
      reference = text1.split()
      candidate = text2.split()

      # Compute BLEU score
      smoothing_function = SmoothingFunction().method1
      bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

      # Compute ROUGE score
      scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
      rouge_scores = scorer.score(text1, text2)

      # Print the results
      print(f"BLEU score: {bleu_score}")
      print(f"ROUGE-1: {rouge_scores['rouge1']}")
      print(f"ROUGE-L: {rouge_scores['rougeL']}")

      print("\n\n")
  # Now let's print the results on the whole corpus.
  print("\n\nTHE RESULT ON THE TWO CORPORA ARE:\n")
  # Tokenize the texts
  reference = results_corpus.split()
  candidate = dataset_corpus.split()

  # Compute BLEU score
  smoothing_function = SmoothingFunction().method1
  bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_function)

  # Compute ROUGE score
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
  rouge_scores = scorer.score(text1, text2)

  # Print the results
  print(f"BLEU score: {bleu_score}")
  print(f"ROUGE-1: {rouge_scores['rouge1']}")
  print(f"ROUGE-L: {rouge_scores['rougeL']}")

## Let's evaluate the results of Mistral.

In [43]:
dataset_path = "prompt_Q2/"
result_path = "mistral_7b_Q2_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.
Sample:  prompt_n_hours=2_skip_hours=318.txt
dev_eui: 124 
 dev_eui: 128


dev_eui: 122, dev_eui: 124, dev_eui: 128
BLEU score: 0.11404605374835301
ROUGE-1: Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8)
ROUGE-L: Score(precision=1.0, recall=0.6666666666666666, fmeasure=0.8)



Sample:  prompt_n_hours=2_skip_hours=390.txt
dev_eui: 124 
 dev_eui: 126 
 dev_eui: 128


dev_eui: 123, dev_eui: 128, dev_eui: 124, dev_eui: 126
BLEU score: 0.07356556000975763
ROUGE-1: Score(precision=1.0, recall=0.75, fmeasure=0.8571428571428571)
ROUGE-L: Score(precision=0.6666666666666666, recall=0.5, fmeasure=0.5714285714285715)



Sample:  prompt_n_hours=2_skip_hours=710.txt
dev_eui: 120 
 dev_eui: 128


dev_eui: 122, dev_eui: 124, dev_eui: 128
BLEU score: 0.11404605374835301
ROUGE-1: Score(precision=0.8333333333333334, recall=0.5555555555555556, fmeasure=0.6666666666666667)
ROUGE-L: Score(precision=0.8333333333333334, recall=0.555555555555555

## Let's evaluate the results of the source code generated by Mistral.

In [ ]:
dataset_path = "prompt_Q1-2024-05-20/"
result_path = "mistral_7b_source_code_results/"
evaluate_results(dataset_path,result_path)

Streaming output truncated to the last 5000 lines.
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 2.
For DEVICE dev_eui:122 all received frames are 1.
For DEVICE dev_eui:123 all received frames are 3.
For DEVICE dev_eui:124 all received frames are 2.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 all received frames are 4.
BLEU score: 1.0
ROUGE-1: Score(precision=1.0, recall=1.0, fmeasure=1.0)
ROUGE-L: Score(precision=1.0, recall=1.0, fmeasure=1.0)



Sample:  prompt_n_hours=2_skip_hours=362.txt
For DEVICE dev_eui:120 all received frames are 2.
For DEVICE dev_eui:121 all received frames are 4.
For DEVICE dev_eui:122 all received frames are 2.
For DEVICE dev_eui:123 all received frames are 2.
For DEVICE dev_eui:124 all received frames are 4.
For DEVICE dev_eui:126 all received frames are 2.
For DEVICE dev_eui:127 all received frames are 2.
For DEVICE dev_eui:128 